# Computational Systems Biology
# HW1

This is a warm-up homework to get familiarized with the COBRA toolbox and basic concepts of optimization in the metabolic networks. This notebook is provided to make your work easier, and it is prepared in the Python language. Still, you can use any other language like Julia (which is perfect for optimization problems) or MATLAB (which also contains a COBRA toolbox). 

## Installing COBRA toolbox

To install the COBRA toolbox, uncomment the script below and run it. You can do it in any other way that you know. If you faced any problem, ask your TA or simply search on google :)

In [ ]:
# import sys
# !$sys.executable -m pip install --upgrade cobra # --ignore-installed ruamel.yaml

# COBRA Tutorial - Building a Model

In this section, you see some useful scripts in the COBRA toolbox and learn how to use them. This material is based in the official COBRA toolbox tutorial at this link: https://cobrapy.readthedocs.io/en/latest/.

## Model, Reactions and Metabolites

This simple example demonstrates how to create a model, create a reaction, and then add the reaction to the model.

In [ ]:
import cobra
from cobra import Model, Reaction, Metabolite

Creating a model is simply done by calling `Model('model_name')`:

In [ ]:
model = Model('example_model')

We need to create metabolites of the model. If we were using an existing model, we could use `Model.get_by_id` to get the appropriate Metabolite objects. Otherwise, we create a metabolite in this format:
`Metabolite('id', 'formula', 'name', 'compartment')`.

In [ ]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

Compartment `'c'`, represents the cytosol. Other familiar compartments are `'e'` (extracellular) and `'p'` (periplasm).

As for the reaction, We'll use a reaction named '3OAS140':

1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] $\rightarrow$ 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]

First, we should create the reaction in the format of `Reaction('id')` and then assign a `name`, `subsystem`, `lower_bound`, `upper_bound` or any other attributes to it (which also could be set as the argument of `Reaction`):

In [ ]:
reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

**Side note:** It is highly recommended to use the standard SBML (Systems Biology Markup Language) format for the ids for reactions, metabolites, and genes. That is, using `'R_id'` for reaction, `'M_id'` for metabolites, and `'G_id'` for genes. 

**Side note 2:** If no particular information about bounds exists, we use the defaults below: <br>
- Irrevsible forward reaction: $[0, M]$
- Irrevsible backward reaction: $[-M, 0]$
- Revsible reaction: $[-M, M]$
- Exchange reaction for metabolites not in the growth medium: $[0, M]$
- Exchange reactions for metabolites available in excess in the growth medium: $[-M, M]$
- Reactions that cannot carry any flux due to the regulatory constraints: $[0, 0]$

in which $M$ is a sufficiently big number, usually set to $1000$ by default. Besides, for at least one exchange reaction for a metabolite existing in a limited concentration in the growth medium (usually the carbon source), we should have the bounds $[-c, M]$, by c representing the limiting flux for such exchange reaction. (otherwise, the optimization problem would be trivial)

Adding metabolites to a reaction uses a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [ ]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

In [ ]:
reaction.reaction  # This gives a string representation of the reaction

It is possible to add genes data to the model. The gene_reaction_rule is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308). We will assign the gene reaction rule string, which will automatically create the corresponding gene objects.

In [ ]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction.genes

We can see the model reactions, metabolites, and genes by calling them like `model.reactions` and so on.  
At this point in time, the model is still empty

In [ ]:
print(f'{len(model.reactions)} reactions initially')
print(f'{len(model.metabolites)} metabolites initially')
print(f'{len(model.genes)} genes initially')

We will add the reaction to the model, which will also add all associated metabolites and genes

In [ ]:
model.add_reactions([reaction])

# The objects have been added to the model
print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')

Note that a reaction could be removed from the model by using `model.remove_reactions([reaction])`.

We can iterate through the model objects to observe the contents

In [ ]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

## Objective

Last we need to set the objective of the model. Here, we just want this to be the maximization of the flux in the single reaction we added and we do this by assigning the reaction's identifier to the `objective` property of the model.

In [ ]:
model.objective = 'R_3OAS140'

The created objective is a symbolic algebraic expression and we can examine it by printing it

In [ ]:
print(model.objective.expression)
print(model.objective.direction)

which shows that the solver will maximize the flux in the forward direction and could be changed to a minimization problem by using `model.objective.direction = 'min'`.

## Tailored Constraints

To add an arbitrary constraint on fluxes, else than stoichiometric and bounds constraints, first, you should make a constraint object:  
`new_constraint = model.problem.Constraint(rxn.flux_expression, lb=min_val, ub=max_val)`,  <br>
which states that the flux of the reaction `rxn` should be between values `min_val` and `max_val`. You can replace 
`rxn.flux_expression` with a weighted sum of many fluxes. For example:
`rxn1.flux_expression - 3 * rxn2.flux_expression`.  <br>
Finally, you need to add this constraint to your model by using `model.add_cons_vars(new_constraint)`. <br>
Note: adding non-linear constraints is also possible, but you should be aware that your problem is no longer an LP.

## Model Validation

For exchange with other tools you can validate and export the model to SBML.
For more information on serialization and available formats see the relative sections in the COBRA documentation. 

In [ ]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

The model is valid with no COBRA or SBML errors or warnings.

## Boundary Reactions: Exchanges, Sinks and Demands
Boundary reactions can be added using the model's method `add_boundary`.
There are three different types of pre-defined boundary reactions: exchange, demand, and sink reactions. All of them are unbalanced pseudo reactions, which means they fulfill a function for modeling by adding to or removing metabolites from the model system but are not based on real biology. An exchange reaction is a reversible reaction that adds to or removes an extracellular metabolite from the extracellular compartment (mostly, we use this kind). A demand reaction is an irreversible reaction that consumes an intracellular metabolite (and is rarely used). A sink is similar to an exchange but specifically for intracellular metabolites, i.e., a reversible reaction that adds or removes an intracellular metabolite (primarily for biomass reaction).

In [ ]:
print("exchanges", model.exchanges)
print("demands", model.demands)
print("sinks", model.sinks)

Boundary reactions are defined on metabolites. First we add two metabolites to the model then
we define the boundary reactions. We add glycogen to the cytosolic compartment `c` and CO2 to the external compartment `e`.

In [ ]:
model.add_metabolites([
    Metabolite(
    'glycogen_c',
    name='glycogen',
    compartment='c'
    ),
    Metabolite(
    'co2_e',
    name='CO2',
    compartment='e'
    ),
])

In [ ]:
# create exchange reaction
model.add_boundary(model.metabolites.get_by_id("co2_e"), type="exchange")

As you see, the exchange reaction for a metabolite with id `'M'` would have the id `'EX_M'` and could be accessible by `model.reactions.get_by_id("EX_M")` for making appropriate changes on its attributions, such as bounds.

In [ ]:
# create exchange reaction
model.add_boundary(model.metabolites.get_by_id("glycogen_c"), type="sink")

Similarly, the sink reaction for a metabolite with id `'M'` would have the id `'SK_M'` and could be accessible by `model.reactions.get_by_id("SK_M")`.

In [ ]:
# Now we have an additional exchange and sink reaction in the model
print("exchanges", model.exchanges)
print("sinks", model.sinks)
print("demands", model.demands)

To create a demand reaction instead of a sink use type `demand` instead of `sink`.

Aggregated information on all boundary reactions is available via the model's property `boundary`.

In [ ]:
# boundary reactions
model.boundary

Note that all boundary reactions are included in `model.reactions`. A neat trick to get all metabolic reactions is

In [ ]:
# metabolic reactions
set(model.reactions) - set(model.boundary)

## Running FBA

Simulations using flux balance analysis (FBA) can be solved using `Model.optimize()`. This will maximize or minimize (maximizing is the default) flux through the objective reactions.

In [ ]:
solution = model.optimize()

This solution is an object by four parts, as bellow:

In [ ]:
print(solution.status)  # Optimal, Infeasible, ...
print(solution.objective_value)  # The optimal value for the objective function
print(solution.fluxes)  # The value for all fluxes in this optimal solution (causion: not unique!)
print(solution.shadow_prices)  # The value for dual variables

Also, `model.summary()` will show some details about the optimal solution:

In [ ]:
model.summary()

# Task 1:

## Part 1:

Build the metabolic network as described in this picture, using the default bound and $D$ as the biomass reaction. Assume that $A$ is the limiting resource allowed to be taken up with a maximum flux of $10$, $B$ is an unlimited resource, and $F$ is the exported metabolic product. Afterward, find the maximum biomass flux rate in this network.

![task1_img](./Fig_3,4.png)

After running FBA, save you model using   
`cobra.io.write_sbml_model(your_model, "./Solutions/Task 1/task1_part1.xml")`.

In [ ]:
## Your Code

## Part 2:

Consider that we have distinguished that our network has some flaws (can you understand it by the result of your FBA?). After studying, we perceived that a new metabolite $G$ was contributing to the $rxn2$, i.e., $C[c] \leftrightarrow F[c] + G[c]$. Also, $G$ could be converted to the metabolite $A$ or vice versa. Modify your model, and run FBA on it to find the optimum value.

![task1_2_img](./Fig_3,4_2.png)

After running FBA, save you model, using  
`cobra.io.write_sbml_model(your_model, "./Solutions/Task 1/task1_part2_model.xml")`.

In [ ]:
## Your Code

# Running FBA on an existing organism's metabolic network

## Importing metabolic data

In your HW folder, a file named "iAF1260.xml" is placed and contains the metabolic network for the well-known organism E.Coli, in the SBML format. This script helps you to read it:

In [ ]:
task2_model = cobra.io.read_sbml_model("iAF1260.xml")

In this model, the objective biomass reaction is preset to `"Ec_biomass_iAF1260_WT_59p81M"`, and all boundaries for reactions are set to standards according to the biological reversibilities in the model. For exchange reactions, all have an upper-bound equal to $M=1000$. The lower bounds are set to $0$, else for the metabolites available in the growth medium. In our special medium, called the M9 medium, there exists a limited glucose uptake, which is demonstrated by lower-bound(EX_glc(e)) = $-10$ in the model. Also, as we have assumed to be in an aerobic condition, the oxygen uptake has been set to $20$ (i.e., lower-bound(EX_o2(e)) = $-20$). Other lower-bounds for external reactions are set to $-M$, as there are supposed to be available in excess in our medium. You don't need to change any of these bounds in this section, but as extra information, there are some reactions in the aerobic condition which cannot carry any flux due to the regulatory constraints and are presented in the `"iAF1260_off_reactions_aero.txt"` file. Be careful to set their fluxes to zero in the aerobic condition.

# Task 2: 

Run FBA on this model and find the maximum biomass flux. As you know, the FBA optimal fluxes are rarely unique and are consequently unreliable. Find the reactions whose fluxes are reliable in the FBA solution. If necessary, consider the precious of the zero to be `1E-6` (i.e., every $\epsilon \in [-1E-6,+1E-6]$ is considered zero).

**Output format:** Make a dictionary with keys to be the ids for reactions with reliable fluxes in FBA and values to be their reliable flux in FBA (i.e., `{..., 'rxn_id': flux, ...}`) and save it using `json.dump`.

In [ ]:
## Your Code

In [ ]:
## Save Your Result:
import json
with open('./Solutions/Task 2/task2.json', 'w') as fp:
    json.dump(reliables_dict, fp)

# Task 3:

The model included in `"iAF1260.xml"` corresponds to aerobic condition (i.e., oxygen uptaking is possible). By setting the oxygen uptake to zero, run FBA for the anaerobic condition and compare the maximum biomass flux in this condition. Note that the reactions identified in `"iAF1260_off_reactions_aero.txt"` are not necessarily off in anaerobic conditions.

In [ ]:
task3_model = cobra.io.read_sbml_model("iAF1260.xml")  

In [ ]:
## Your Code

In [ ]:
# Save Your Model:
cobra.io.write_sbml_model(task3_model, "./Solutions/Task 3/task3.xml")

# Task 4: 

## Modifying a problem

A researcher is looking for some organism to produce industrial "ethanol" in the lab. He has found that our E.Coli organism iAF1260 is best for that job as solving the following optimization problem has revealed a considerable amount of possible ethanol production under the aerobic condition: <br>
(include off reactions before running)

In [ ]:
task4_model = cobra.io.read_sbml_model("iAF1260.xml") 

In [ ]:
# ... (a lost part of the researcher's work for including off reactions in aerobic condition :)) ) ...

In [ ]:
task4_model.objective = 'EX_etoh(e)'
solution = task4_model.optimize()
print(solution.status) 
print(solution.objective_value) 
print(solution.fluxes) 
print(solution.shadow_prices)  
task4_model.summary()

But when he experimetally cultivated E.Coli in the lab, only an insignificant amount of ethanol was prodeced. 

## Part 1:

Do you see any problem with his computational work? If so, analyze the issue and correct it. <br>
After doing optimization, save your model using `cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part1.xml")`.

In [ ]:
## Your Code

## Part 2:

Do you offer him to treat this experiment under the anaerobic condition? Test it, and after doing optimizations, save your model using `cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part2.xml")`.

In [ ]:
task4_model = cobra.io.read_sbml_model("iAF1260.xml") 

In [ ]:
## Your Code

In [ ]:
# Save Your Model
cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part2.xml")

################################################################################################